# Armazéns Hanoi Simulation: Experimental Results Analysis

This notebook loads, aggregates, and visualizes the results of all simulation experiments. It provides code and visualizations for interpreting the computational and operational performance of the system across all experimental scenarios.

In [6]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import os

# Set up matplotlib for inline plotting
%matplotlib inline

## Load and Aggregate Experimental Results

This section loads all `metrics.json` files from the results directory, aggregates them into a single DataFrame, and extracts experiment parameters for analysis.

In [7]:
# Load all metrics.json files and aggregate results
results_dir = os.path.join('..', 'results')
metrics_files = glob.glob(os.path.join(results_dir, '*', 'metrics.json'))
meta_files = [f.replace('metrics.json', os.path.basename(f).replace('metrics.json', 'in.txt.meta.json')) for f in metrics_files]

records = []
for mfile, metafile in zip(metrics_files, meta_files):
    with open(mfile) as f:
        metrics = json.load(f)
    params = {}
    if os.path.exists(metafile):
        with open(metafile) as mf:
            meta = json.load(mf)
            params = meta.get('params', {})
    record = {**params, **metrics, 'metrics_file': mfile}
    records.append(record)

df = pd.DataFrame(records)
print(f"Loaded {len(df)} experiment results.")
df

Loaded 56 experiment results.


,avg_section_depth,delivery_time_stats,execution_time,heap_extracts,heap_inserts,max_section_depth,packages_moved,peak_memory_kb,re_storage_events,stack_pops,stack_pops_rearmazenado,stack_pushes,storage_time_stats,transit_time_stats,transport_capacity,transport_events,transport_utilization,metrics_file
0,3.577288e+07,"{'max': 526, 'mean': 245.958, 'median': 251, '...",0.322198,3002,3019,858862860,2740,3548,3,1370,0,1370,"{'max': 427, 'mean': 94.29041697147038, 'media...","{'max': 179, 'mean': 69.96269202633503, 'media...",5,1117,49.059982,../results/baseline.in/metrics.json
1,3.577288e+07,"{'max': 526, 'mean': 245.958, 'median': 251, '...",0.309082,3002,3019,858862860,2740,3496,3,1370,0,1370,"{'max': 427, 'mean': 94.29041697147038, 'media...","{'max': 179, 'mean': 69.96269202633503, 'media...",5,1117,49.059982,../results/expA_10armazens.in/metrics.json
2,1.356035e+08,"{'max': 821, 'mean': 336.18333333333334, 'medi...",0.469001,4536,4557,1171555798,4366,3764,8,2183,0,2183,"{'max': 722, 'mean': 165.64183908045976, 'medi...","{'max': 181, 'mean': 72.74022988505747, 'media...",5,1739,50.212766,../results/expA_12armazens.in/metrics.json
3,6.399732e+07,"{'max': 716, 'mean': 309.91333333333336, 'medi...",0.587452,5993,6020,858862860,5068,3660,8,2534,0,2534,"{'max': 616, 'mean': 134.44893111638956, 'medi...","{'max': 180, 'mean': 72.01702296120348, 'media...",5,2689,37.694310,../results/expA_15armazens.in/metrics.json
4,1.580298e+08,"{'max': 821, 'mean': 368.5270588235294, 'media...",0.756099,7604,7635,858862863,6748,3756,13,3374,0,3374,"{'max': 722, 'mean': 175.59714370723, 'median'...","{'max': 180, 'mean': 73.20083308539125, 'media...",5,3361,40.154716,../results/expA_17armazens.in/metrics.json
5,2.650013e+15,"{'max': 819, 'mean': 359.15222222222224, 'medi...",0.826790,8185,8218,18446744072904412629,6970,3692,9,3485,0,3485,"{'max': 721, 'mean': 164.66398158803221, 'medi...","{'max': 180, 'mean': 72.99108170310701, 'media...",5,3775,36.927152,../results/expA_18armazens.in/metrics.json
6,3.508865e+08,"{'max': 910, 'mean': 399.624, 'median': 382, '...",0.977162,10099,10136,1397863958,8568,4116,12,4284,0,4284,"{'max': 810, 'mean': 191.46020599250937, 'medi...","{'max': 179, 'mean': 73.54494382022472, 'media...",5,4789,35.782000,../results/expA_20armazens.in/metrics.json
7,4.084038e+08,"{'max': 1117, 'mean': 462.67833333333334, 'med...",1.426988,14406,14451,2140478869,11796,4256,7,5898,0,5898,"{'max': 1017, 'mean': 230.98455270751995, 'med...","{'max': 179, 'mean': 74.24783568154812, 'media...",5,7269,32.455634,../results/expA_24armazens.in/metrics.json
8,1.004049e+08,"{'max': 1383, 'mean': 547.7857142857143, 'medi...",1.830576,19237,19290,858862862,16148,4292,12,8074,0,8074,"{'max': 1283, 'mean': 288.9181344579509, 'medi...","{'max': 177, 'mean': 75.12527908707517, 'media...",5,9721,33.222919,../results/expA_28armazens.in/metrics.json
9,3.289055e+08,"{'max': 1701, 'mean': 658.504375, 'median': 60...",2.560060,25175,25236,1577015159,22032,4600,28,11016,0,11016,"{'max': 1601, 'mean': 383.09765198398253, 'med...","{'max': 181, 'mean': 75.88705860939207, 'media...",5,12525,35.180838,../results/expA_32armazens.in/metrics.json


## Visualize Key Metrics

This section provides visualizations for the main metrics across all experiments:
- Execution time, memory usage, heap/stack operations
- Package delivery times (mean, median, 95th percentile, max)
- Re-storage events, warehouse section depth, transport utilization
- Line plots, histograms, and heatmaps as described in the experimental plan

In [ ]:
# Example: Execution Time vs. Number of Warehouses (Experiment A)
expA = df[df['metrics_file'].str.contains('expA_')]
plt.figure(figsize=(8,5))
plt.plot(expA['numeroarmazens'], expA['execution_time'], marker='o')
plt.xlabel('Number of Warehouses')
plt.ylabel('Total Execution Time (s)')
plt.title('Experiment A: Execution Time vs. Number of Warehouses')
plt.grid(True)
plt.show()

In [ ]:
# Example: Histogram of Package Delivery Times (select an experiment)
example = df.iloc[0]  # Change index to select different experiment
if 'delivery_time_stats' in example and isinstance(example['delivery_time_stats'], dict):
    stats = example['delivery_time_stats']
    # If you have raw delivery times, plot histogram
    # Otherwise, plot summary stats
    plt.figure(figsize=(6,4))
    plt.bar(['mean', 'median', 'p95', 'max'], [stats.get(k, 0) for k in ['mean','median','p95','max']])
    plt.title('Package Delivery Time Stats')
    plt.ylabel('Time')
    plt.show()
else:
    print('No delivery time stats available in this result.')

In [ ]:
# Example: Heatmap for Experiment C (capacity vs. interval, color = re-storage events)
expC = df[df['metrics_file'].str.contains('expC_')]
heatmap_data = expC.pivot_table(index='capacidadetransporte', columns='intervalotransportes', values='re_storage_events', aggfunc='mean')
plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Experiment C: Re-storage Events Heatmap')
plt.xlabel('Transport Interval')
plt.ylabel('Transport Capacity')
plt.show()

## Interpretation and Summary of Findings

Use the visualizations above to interpret the results for each experiment:
- Identify trends, bottlenecks, and inflection points
- Relate observed behaviors to the underlying data structures and algorithms
- Summarize key findings and recommendations for scaling the Armazéns Hanoi system

*Fill in your analysis and conclusions below.*